In [1]:
!git clone https://github.com/openai/shap-e.git
%cd shap-e
!pip install -e .


fatal: destination path 'shap-e' already exists and is not an empty directory.
/content/shap-e
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-b3pgb324/clip_a10acbb37c3e48f8911a6a84b0522c01
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-b3pgb324/clip_a10acbb37c3e48f8911a6a84b0522c01
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Attempting uninstall: shap-e
    Found existing installation: shap-e 0.0.0
    Uninstalling shap-e-0.0.0:
      Successfully uninstalled shap-e-0.0.0
  Running setup.py develop for shap-e


In [15]:
!pip install -q rembg onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00


In [16]:

import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, decode_latent_mesh,gif_widget
from shap_e.util.image_util import load_image
from datetime import datetime
from rembg import remove
from PIL import Image
from io import BytesIO

In [3]:
# Load models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
modelTxt = load_model('text300M', device=device)
modelImg = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/content/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [19]:
def remove_background(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    output = remove(img_byte_arr)
    result_image = Image.open(BytesIO(output))
    white_bg = Image.new('RGB', result_image.size, (255, 255, 255))
    if result_image.mode == 'RGBA':
        white_bg.paste(result_image, mask=result_image.split()[3])
    else:
        white_bg.paste(result_image)
    return white_bg

In [30]:
def generate_image_3d(image_p):
  batch_size = 4
  guidance_scale = 3.0

  image = load_image(image_p)
  image = remove_background(image)

  latents = sample_latents(
    batch_size=batch_size,
    model=modelImg,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
    )
  render_mode = 'nerf' #  'stf' for mesh rendering
  size = 128 #64
  cameras = create_pan_cameras(size, device)
  for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [33]:
def generate_text_3d(prompt):
  batch_size = 3
  guidance_scale = 15.0
  prompt = prompt

  latents = sample_latents(
    batch_size=batch_size,
    model=modelTxt,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
    )
  render_mode = 'nerf' # 'stf'
  size = 128
  cameras = create_pan_cameras(size, device)
  for i, latent in enumerate(latents):
      images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
      display(gif_widget(images))

  from shap_e.util.notebooks import decode_latent_mesh

  for i, latent in enumerate(latents):
      t = decode_latent_mesh(xm, latent).tri_mesh()
      with open(f'shapE_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
      with open(f'ShapE_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)


In [35]:
generate_text_3d("a tree")  # Text input


  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAABxUGxtSGhtRGhtPGhtOGhlOGRpNGBpLGBhMFxpKFxdJFxlHFh…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAADxNJTFNIDBOHzBKHy9MHy9JHi5MHy5KHi5IHi1MIC1LHy1KHy…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAG1PPD9SFj1UEj5PFTxUEjxPFDtSEjtQFDtPFTpPFTpPFDpPEj…

In [ ]:
generate_image_3d("/content/imageName")  # Image input